In [4]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from openai import OpenAI
import os

%load_ext dotenv
%dotenv
OPENAI_API_KEY = %env OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)

/Users/alexfroitzheim/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
data_dir = '../data/with-wiki/'
out_dir = '../data/annotated/'

In [6]:
# read prompt
with open("../prompts/prompt_v1.txt", "r") as f:
    prompt = f.read()


def row_to_prompt(row, is_first):
    fever_part = f'CLAIM: {row["claim"]}'
    ev_sents = sent_tokenize(row["evidence_text"])

    for i, sent in enumerate(ev_sents):
        fever_part += f"\n{i + 1}: {sent}"

    if is_first:
        return prompt + fever_part
    else:
        return fever_part


def get_annotation(row):
    curr_prompt = row_to_prompt(row, True)

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": curr_prompt},
        ],
    )

    return completion.choices[0].message.content

def add_annotations(df):
    df["annotation"] = df.apply(get_annotation, axis=1)
    return df

In [19]:
# for each file in data dir, read to df, add annotations, save to new file
for filename in os.listdir(data_dir):
    print(f"Reading {filename}")
    df = pd.read_csv(os.path.join(data_dir, filename), sep=';')
    df = df[df['evidence_text'].notna()]
    df = add_annotations(df)
    df.to_csv(os.path.join(out_dir, filename.replace(".csv", "_annotated.csv")), sep=';')

Reading df_with_text_800.csv
Reading df_with_text_0.csv
Reading df_with_text_900.csv
Reading df_with_text_500.csv
Reading df_with_text_700.csv
Reading df_with_text_300.csv
Reading df_with_text_100.csv
Reading df_with_text_600.csv
Reading df_with_text_400.csv
Reading df_with_text_200.csv
